# Extracting dates from medical records
### - Aashita Kesarwani

The aim of the project is first to identify dates given in different formats from the messy medical data and convert them in the same format and then sort them. The project was a weekly assignment from the course [Applied Text Mining in Python](https://www.coursera.org/learn/python-text-mining/) offered by the University of Michigan on Coursera. 

First, we start with loading data and importing pandas module for working with the data:

In [1]:
import pandas as pd
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)

Our data consists of medical notes where each note has a date that needs to be extracted. There are 500 records in total as shown below:

In [2]:
df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

An example of medical record is shown below. 

In [3]:
df[19]

') 59 yo unemployed w referred by Urgent Care for psychiatric evaluation and follow up. The patient reports she has been dx w BAD. Her main complaint today is anergy. Ms. Hartman was evaluated on one occasion 5/21/77. She was a cooperative but somewhat vague historian.History of Present Illness and Precipitating Events\n'

The dates in the records are encoded in different formats. Here is a list of some of the variants present in this dataset:  
1. 04/20/2009; 04/20/09; 4/20/09; 4/3/09  
2. Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009  
3. 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009  
4. Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009  
5. Feb 2009; Sep 2009; Oct 2010  
6. 6/2008; 12/2009  
7. 2009; 2010    
 
I encountered a couple typos in the dates as this is a raw, real-life derived dataset. Below is the code to extract all the dates while ignoring other numerals that might seem similar to dates.

In [4]:
Text = df.copy()
Text = Text.str.replace("Janaury", "January")
Text = Text.str.replace("Decemeber", "December")
def re_1(text):
    return text.str.extract("(?P<Date>(?P<Month>[01]?\d)[/-](?P<Day>[0123]?\d)[/-](?P<Year>\d{4}))")
def re_2(text):
    return text.str.extract("(?P<Date>(?P<Month>[01]?\d)[/-](?P<Day>[0123]?\d)[/-](?P<Year>\d{2}))")
def re_3(text):
    return text.str.extract("(?P<Date>(?P<Month>[01]?\d)[/-](?P<Year>[12][90]\d{2}))")
def re_4(text):
    return text.str.extract("(?P<Date>(?P<Day>[0123]?\d)\s(?P<Month>January|February|March|April|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(?P<Year>\d{4}))")
def re_5(text):
    return text.str.extract("(?P<Date>(?P<Month>January|February|March|April|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.?\s(?P<Day>[0123]?\d),?\s(?P<Year>\d{4}))")
def re_6(text):
    return text.str.extract("(?P<Date>(?P<Month>January|February|March|April|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec),?\s(?P<Year>\d{4}))")
def re_7(text):
    return text.str.extract("(?P<Date>(?P<Year>\d{4}))")
df1 = re_1(Text).dropna(how="all")
Text = Text.drop(list(df1.index))
df2 = re_2(Text).dropna(how="all")
Text = Text.drop(list(df2.index))
df2.Year = df2.Year.apply(lambda x: '19' + x)
df3 = re_3(Text).dropna(how="all")
df3["Day"] = 1
Text = Text.drop(list(df3.index))
df4 = re_4(Text).dropna(how="all")
df4.Month = df4.Month.apply(lambda x: x[:3])
Text = Text.drop(list(df4.index))
df5 = re_5(Text).dropna(how="all")
df5.Month = df5.Month.apply(lambda x: x[:3])
Text = Text.drop(list(df5.index))
df6 = re_6(Text).dropna(how="all")
df6.Month = df6.Month.apply(lambda x: x[:3])
df6["Day"] = 1
Text = Text.drop(list(df6.index))
df7 = re_7(Text).dropna(how="all")
df7["Day"] = 1
df7["Month"] = 1
Text = Text.drop(list(df7.index))

Date = pd.concat([df1, df2, df3, df4, df5, df6, df7])
Date.sample(30)

,Date,Day,Month,Year
69,11/3/1985,3,11,1985
22,3/03/90,03,3,1990
103,9/22/82,22,9,1982
480,2013,1,1,2013
229,June 2011,1,Jun,2011
259,Nov 1979,1,Nov,1979
320,November 2012,1,Nov,2012
155,10 Oct 1974,10,Oct,1974
310,Oct 1992,1,Oct,1992
331,December 1993,1,Dec,1993


Once these date patterns are extracted from the text, they are encoded in the same format of  the next step is to sort them in ascending chronological order according to the following rules:  
1. Assumed all dates in xx/xx/xx format are mm/dd/yy.    
2. Assumed all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989).  
3. If the day is missing (e.g. 9/2009), assumed it is the first day of the month (e.g. September 1, 2009).    
4. If the month is missing (e.g. 2010), assumed it is the first of January of that year (e.g. January 1, 2010).

Below is the code to sort the dates in ascending chronological order:

In [5]:
months_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
               'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 9, 'Dec': 12}
Date.Month = Date.Month.replace(months_dict)
Date.Month = Date.Month.astype('int')
Date.Day = Date.Day.astype('int')
Date.Year = Date.Year.astype('int')
Date = Date.sort_values(by=["Year", "Month", "Day"])
Date

,Date,Day,Month,Year
9,4/10/71,10,4,1971
84,5/18/71,18,5,1971
2,7/8/71,8,7,1971
53,7/11/71,11,7,1971
28,9/12/71,12,9,1971
474,1972,1,1,1972
153,13 Jan 1972,13,1,1972
13,1/26/72,26,1,1972
129,06 May 1972,6,5,1972
98,5/13/72,13,5,1972


The first column above is the index which is unique to each medical record, so now we have medical records sorted in ascending chronological order of the dates mentioned in them. The indices of the earliest 10 medical records are given below:

In [6]:
print("\n".join(map(str,Date.index[:10])))

9
84
2
53
28
474
153
13
129
98
